# Example 1: RDD

In [ ]:
import pyspark
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
spark: SparkSession = SparkSession.builder \
    .appName("poc") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "localhost") \
    .getOrCreate()


sc: SparkContext = spark.sparkContext

# conf = SparkConf().setAppName("poc") \
#     .setMaster("local[1]")
# conf.set("spark.driver.host", "localhost")
#
# sc = SparkContext(conf=conf)

BASE_PATH = Path().resolve()
BASE_PATH

In [ ]:
path = BASE_PATH / "resource" / "reviews_devset.json"
rdd = sc.wholeTextFiles(str(path))

display(rdd)

# Example 2: 